In [1]:
!unzip archive.zip -d my_data

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: my_data/train/REAL/4500 (10).jpg  
  inflating: my_data/train/REAL/4500 (2).jpg  
  inflating: my_data/train/REAL/4500 (3).jpg  
  inflating: my_data/train/REAL/4500 (4).jpg  
  inflating: my_data/train/REAL/4500 (5).jpg  
  inflating: my_data/train/REAL/4500 (6).jpg  
  inflating: my_data/train/REAL/4500 (7).jpg  
  inflating: my_data/train/REAL/4500 (8).jpg  
  inflating: my_data/train/REAL/4500 (9).jpg  
  inflating: my_data/train/REAL/4500.jpg  
  inflating: my_data/train/REAL/4501 (10).jpg  
  inflating: my_data/train/REAL/4501 (2).jpg  
  inflating: my_data/train/REAL/4501 (3).jpg  
  inflating: my_data/train/REAL/4501 (4).jpg  
  inflating: my_data/train/REAL/4501 (5).jpg  
  inflating: my_data/train/REAL/4501 (6).jpg  
  inflating: my_data/train/REAL/4501 (7).jpg  
  inflating: my_data/train/REAL/4501 (8).jpg  
  inflating: my_data/train/REAL/4501 (9).jpg  
  inflating: my_data/train/REAL/4501.jpg 

# Импорт библиотек

In [6]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.applications.efficientnet import preprocess_input
from tensorflow import keras
from keras.models import load_model

In [13]:
import sys
import PIL
print(f'python== {sys.version}')
print(f'pandas== {pd.__version__}')
print(f'numpy== {np.__version__}')
print(f'tensorflow== {tf.__version__}')
print(f'pillow== {PIL.__version__}')

python== 3.10.12 (main, Mar 22 2024, 16:50:05) [GCC 11.4.0]
pandas== 2.0.3
numpy== 1.25.2
tensorflow== 2.15.0
pillow== 9.4.0


# Константы

In [3]:
IMG_SIZE = 224
DATA_FOLDER = '/content/my_data'
# '/content/drive/MyDrive/Контур/'
# IMG_FOLDER = os.path.join(DATA_FOLDER, 'images/')
TRAIN_PATH = os.path.join(DATA_FOLDER, 'train/')
TEST_PATH = os.path.join(DATA_FOLDER, 'test/')
# IMG_PATH = os.path.join(DATA_FOLDER, 'my_images/')
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.2

# Fine-tuning CNN

## Импорт данных

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  TRAIN_PATH,
  validation_split=VALIDATION_SPLIT,
  subset="training",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE),
  batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
  TRAIN_PATH,
  validation_split=VALIDATION_SPLIT,
  subset="validation",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE),
  batch_size=BATCH_SIZE)

Found 100000 files belonging to 2 classes.
Using 80000 files for training.
Found 100000 files belonging to 2 classes.
Using 20000 files for validation.


## EfficientNet

### Обучение модели

In [5]:
# Building the Transfer Learning model using EfficientNetV2B0
EfficientNet_base_model = tf.keras.applications.EfficientNetV2B0(
    include_top = False,
    weights = 'imagenet',
    input_shape = (IMG_SIZE, IMG_SIZE, 3),
    pooling = 'max'
)
EfficientNet_base_model.trainable = True

# Create a new model on top of the EfficientNet base
inputs = tf.keras.Input(shape = (IMG_SIZE, IMG_SIZE, 3))
x = EfficientNet_base_model(inputs, training = False)
x = BatchNormalization(axis = -1, momentum = 0.99, epsilon = 0.001)(x)
x = Dense(256,
          kernel_regularizer = regularizers.l2(0.01),
          activity_regularizer = regularizers.l1(0.01),
          bias_regularizer = regularizers.l1(0.01),
          activation = 'relu')(x)
x = Dropout(rate = .4, seed = 512)(x)
x = Dense(64, activation = 'relu')(x)
outputs = Dense(1, activation = 'sigmoid')(x)
EfficientNet_model = tf.keras.Model(inputs, outputs)

# Compile the Transfer Learning model
EfficientNet_model.compile(
    optimizer = tf.keras.optimizers.Adamax(learning_rate = .001),
    loss = tf.keras.losses.BinaryCrossentropy()
)

# Build the Transfer Learning model so we can see a summary
EfficientNet_model.summary()

24274472/24274472 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Functio  (None, 1280)              5919312   
 nal)                                                            
                                                                 
 batch_normalization (Batch  (None, 1280)              5120      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                              

In [6]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                  patience = 10,
                                                  restore_best_weights = True,
                                                  min_delta=0.01)

In [7]:
# with augmantation & normalization
EfficientNet_model_history = EfficientNet_model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 100,
    verbose = 1,
    callbacks = [early_stopping]
)

Epoch 1/100
5000/5000 [==============================] - 611s 112ms/step - loss: 0.5439 - val_loss: 0.3255
Epoch 2/100
5000/5000 [==============================] - 566s 113ms/step - loss: 0.1800 - val_loss: 0.1181
Epoch 3/100
5000/5000 [==============================] - 559s 112ms/step - loss: 0.1499 - val_loss: 0.4294
Epoch 4/100
5000/5000 [==============================] - 557s 111ms/step - loss: 0.1433 - val_loss: 0.1469
Epoch 5/100
5000/5000 [==============================] - 554s 111ms/step - loss: 0.1228 - val_loss: 0.6682
Epoch 6/100
5000/5000 [==============================] - 561s 112ms/step - loss: 0.1129 - val_loss: 0.1664
Epoch 7/100
5000/5000 [==============================] - 554s 111ms/step - loss: 0.1009 - val_loss: 0.1748
Epoch 8/100
5000/5000 [==============================] - 555s 111ms/step - loss: 0.0869 - val_loss: 0.1207
Epoch 9/100
5000/5000 [==============================] - 557s 111ms/step - loss: 0.0811 - val_loss: 0.1303
Epoch 10/100
5000/5000 [=============

### Оценка модели

In [8]:
pred_ds = tf.keras.utils.image_dataset_from_directory(
  TEST_PATH,
  # labels=None,
  seed = 512,
  image_size = (IMG_SIZE, IMG_SIZE),
  batch_size = BATCH_SIZE, shuffle=False)

Found 20000 files belonging to 2 classes.


In [9]:
EfficientNet_model.evaluate(pred_ds)

1250/1250 [==============================] - 39s 31ms/step - loss: 0.1186


0.1185634657740593

In [11]:
!unzip train.zip -d train_data

Archive:  train.zip
   creating: train_data/train/
   creating: train_data/train/FAKE/
  inflating: train_data/train/FAKE/0bAvf1FcUP.jpeg  
  inflating: train_data/train/FAKE/0xvbLyG0L2.png  
  inflating: train_data/train/FAKE/1EkEeqlGtK.png  
  inflating: train_data/train/FAKE/1Q2t43zBFt.jpeg  
  inflating: train_data/train/FAKE/1vt2wUUltu.jpeg  
  inflating: train_data/train/FAKE/26uRI1HzsF.jpeg  
  inflating: train_data/train/FAKE/2ap0mXNKe7.jpeg  
  inflating: train_data/train/FAKE/37dJN143sv.jpeg  
  inflating: train_data/train/FAKE/3CwnBMp1Gw.jpeg  
  inflating: train_data/train/FAKE/3YMwaR2miM.jpeg  
  inflating: train_data/train/FAKE/47apyfkXUs.png  
  inflating: train_data/train/FAKE/4CTRMzDhq9.png  
  inflating: train_data/train/FAKE/4DeuZpVM3V.jpeg  
  inflating: train_data/train/FAKE/4e1W3vSwyN.jpeg  
  inflating: train_data/train/FAKE/4M57bqn74c.png  
  inflating: train_data/train/FAKE/4MrAeXsHPf.jpeg  
  inflating: train_data/train/FAKE/4ne2OyoWkB.jpeg  
  inflating: trai

In [12]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/train_data',
  # labels=None,
  seed = 512,
  image_size = (IMG_SIZE, IMG_SIZE),
  batch_size = BATCH_SIZE, shuffle=False)

Found 1012 files belonging to 1 classes.


In [13]:
EfficientNet_model.evaluate(val_ds)

64/64 [==============================] - 8s 123ms/step - loss: 0.0536


0.05358051881194115

### Сохранение модели

#### В формате tf

In [17]:
EfficientNet_model.save('EfficientNet', save_format='tf')

In [18]:
!zip -r /content/EfficientNet.zip /content/EfficientNet

  adding: content/EfficientNet/ (stored 0%)
  adding: content/EfficientNet/fingerprint.pb (stored 0%)
  adding: content/EfficientNet/variables/ (stored 0%)
  adding: content/EfficientNet/variables/variables.data-00000-of-00001 (deflated 6%)
  adding: content/EfficientNet/variables/variables.index (deflated 77%)
  adding: content/EfficientNet/saved_model.pb (deflated 91%)
  adding: content/EfficientNet/keras_metadata.pb (deflated 96%)
  adding: content/EfficientNet/assets/ (stored 0%)


#### В формате h5

In [ ]:
EfficientNet_model.save('EfficientNet.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Предсказание

In [ ]:
EfficientNet_model = load_model(os.path.join(DATA_FOLDER, 'EfficientNet.h5'))

### Для предсказания тестового набора данных

In [ ]:
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))
test['new_id'] = test.id.str.split('.').str.get(0)
image_list = os.listdir(TEST_PATH)
data_image = pd.DataFrame(image_list, columns = ['name_ext'])
data_image['name'] = data_image.name_ext.str.split('.').str.get(0)
test = test.merge(data_image, how='left', left_on = 'new_id', right_on='name')

In [ ]:
pred_ds = tf.keras.utils.image_dataset_from_directory(
  TEST_PATH,
  labels=None,
  seed = 512,
  image_size = (IMG_SIZE, IMG_SIZE),
  batch_size = BATCH_SIZE, shuffle=False)

In [ ]:
filenames = pred_ds.file_paths
file_names = []

# Iterate through all image files in the specified directory
for file in filenames:
    label = str(file).split('/')[-1]  # Extract the label from the file path
    file_names.append(label)

In [ ]:
predictions = EfficientNet_model.predict(pred_ds)
predictions = predictions[:, 0]
pred_df = pd.DataFrame(list(zip(file_names, predictions)), columns = ['Name', 'target'])

32/32 [==============================] - 18s 451ms/step


In [ ]:
res = test.merge(pred_df, left_on='name_ext', right_on = 'Name', how='left')
res = res[['id', 'target']]
res = res[res.target.isna()==False]
res.target = res.target.apply(lambda x: 1-x)

In [ ]:
res.to_csv('predictions.csv', index=False)

### Для предсказания одной картинки

In [ ]:
# Загрузка изображения
img = load_img(os.path.join(IMG_PATH, '0a1KuiILS4.jpg'),
               target_size=(IMG_SIZE, IMG_SIZE))  # Указать путь к изображению и размер

# Преобразование изображения в массив numpy
img_array = img_to_array(img)
# Расширение размерности на 1, чтобы модель могла обработать
img_array = np.expand_dims(img_array, axis=0)
# Предварительная обработка массива
processed_img = preprocess_input(img_array)

predictions = EfficientNet_model.predict(processed_img)
if predictions < 0.5:
  print('Я думаю, что эта картинка сгенерирована нейросетью')
else:
  print('Я думаю, что эта картинка сделана человеком')

1/1 [==============================] - 6s 6s/step
Я думаю, что эта картинка сделана человеком
